In [38]:
import os
import librosa
import librosa.display
#matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import IPython.display
import random
import numpy as np
import tensorflow as tf
import pandas as pd
import time
from sklearn.metrics import confusion_matrix
from datetime import timedelta
import scipy.misc
from tensorflow.python.platform import gfile

In [39]:
PATH_TRAIN = '/Users/hormone03/Desktop/word_recognition/Dataset2/train'
PATH_TEST = '/Users/hormone03/Desktop/word_recognition/Dataset2/test'
BATCH_SIZE = 100
ITERATIONS = 10000
ITERATIONS_TEST = 10
EVAL_EVERY = 5
HEIGHT = 20
WIDTH = 44
NUM_LABELS = 0
LEARNING_RATE = 1E-4
LOGDIR = 'log/'
TEST_LOGDIR = 'log_test/'
LABEL_TO_INDEX_MAP = {}

In [40]:
#Note that u made changes on this section:  
             #max_pool1 = tf.nn.max_pool(drop2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
            #changes made: u chnaged drop1 to drop2 in above line

In [41]:
# Create a mapping from labels to index and vice versa
def init(path):
    labels = os.listdir(path)
    index=0
    for label in labels:
        LABEL_TO_INDEX_MAP[label] = index
        index +=1
    
    global NUM_LABELS
    NUM_LABELS = len(LABEL_TO_INDEX_MAP) # Returns a list of files that match the given pattern(s).

In [42]:
init(PATH_TRAIN)
print(LABEL_TO_INDEX_MAP)



{'cat': 0, 'tree': 1, 'bed': 2, 'dog': 3, 'bird': 4, 'house': 5}


In [43]:
# Function to convert word label to vector
def one_hot_encoding(label):
    encoding =[0] * len(LABEL_TO_INDEX_MAP)
    encoding[LABEL_TO_INDEX_MAP[label]] = 1
    return encoding

In [44]:
# Function to get mel Frequency Ceptrum coefficients 
def get_mfcc(wave_path, PAD_WIDTH=WIDTH):
    wave, sr = librosa.load(wave_path, mono=True) #mono will return 512 samples by default
    mfccs = librosa.feature.mfcc(y=wave, sr=sr, n_mfcc=HEIGHT) 
    #The output of the above function is the matrix mfcc, which is a numpy.ndarray of size (n_mfcc, T)
    # Where T is track duration in frames
    mfccs = np.pad(mfccs, ((0,0), (0, PAD_WIDTH - len(mfccs[0]))), mode='constant') #check ur note for Zero-padding
    return mfccs

In [45]:
# function to get next batch of the labels and audio features
def get_batch(batch_size, path):
    X = []
    Y = []
    random.seed(5896)
    path = os.path.join(path, '*', '*.wav')
    waves = gfile.Glob(path) # Globe Returns a list of files that match the given pattern(s).
    while True:
        random.shuffle(waves)
        for wave_path in waves: #wave_path is like index i
            _,label = os.path.split(os.path.dirname(wave_path)) #wave_path is like index i
            X.append(get_mfcc(wave_path))
            Y.append(one_hot_encoding(label))
            if (len(X) == batch_size):
                yield X, Y
                X = []
                Y = []

In [46]:
def get_model(input, dropout):
    # Topology
    # 1. Convolution Layer
    #       Relu Activation
    #       Dropout
    #       Max Pool
    # 2. Convolutional Layer
    #       Relu Activation
    #       Dropout
    # 3. Fully Connected Layer
    #       MatNul
    
    # First Convolutional Layer
    with tf.name_scope('Conv1'):
        input_4D = tf.reshape(input, [-1, HEIGHT, WIDTH, 1]) # Check your note for explanation 1-> one_channel
        #Why reshape to -1 -1 is a placeholder that says "adjust as necessary to match the size needed for 
        #the full tensor." It's a way of making the code be independent of the input batch size, so that you can 
        #change your pipeline and not have to adjust the batch size everywhere in the code.
        w1 = tf.Variable(tf.truncated_normal([12, 8, 1, 44], stddev=0.01), name="W")
        ###### ([f_height, f_width, input_size->"no_channel", filter_no])
        #Weight1:  tf.Variable(tf.truncated_normal([*filter_size, bottom_channel, dilation_rate, filter_num], dtype=tf.float32, stddev=1e-1)
        b1 = tf.Variable(tf.zeros([44]), name="B") # Biases intialization
        conv1 = tf.nn.conv2d(input_4D, w1, strides=[1, 1, 1, 1], padding="SAME") #[1, x, y, 1]striding along x and y by just 1, no skipping 
        act1 = tf.nn.relu(conv1 + b1) #activation function plu biases
        drop1 = tf.nn.dropout(act1, dropout)
        max_pool1 = tf.nn.max_pool(drop1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")# check your note
        #tf.summary.histogram("wights", w1) ################## ("Names", values)
       #tf.summary.histogram("biases", b1)
        #tf.summary.histogram("activations", act1)
        #tf.summary.histogram("dropouts", drop1)
        
    # Second Convolution Layer
        with tf.name_scope('Conv2'):
            w2 = tf.Variable(tf.truncated_normal([6, 4, 44, 44], stddev=0.01), name="W")
            ###### ([f_height, f_width, input_size->"4rm_previous_layer's_output", filter_no])
            b2 = tf.Variable(tf.zeros([44]), name="B")
            conv2 = tf.nn.conv2d(max_pool1, w2, strides=[1, 1, 1, 1], padding="SAME")
            act2 = tf.nn.relu(conv2 + b2)
            drop2 = tf.nn.dropout(act2, dropout)
            max_pool1 = tf.nn.max_pool(drop2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
            #changes made: u chnaged drop1 to drop2 in above line
            #tf.summary.histogam("wights", w2)
            #tf.summary.histogram("biases", b2)
            #tf.summary.histogram("activations", act2)
            #tf.summary.histogram("dropouts", drop2)

        # Reshaping for Fully Connected layer
            conv_shape = drop2.get_shape()
            count = int(conv_shape[1] * conv_shape[2] * conv_shape[3])
            flat_output = tf.reshape(drop2,[-1, count])

        # Fully Connected Layer
            w3 = tf.Variable(tf.truncated_normal([count, NUM_LABELS], stddev=0.01))
            b3 = tf.Variable(tf.zeros([NUM_LABELS]))
            fc = tf.add(tf.matmul(flat_output, w3), b3)
            #tf.summary.histogram("weights", w3)
            #tf.summary.histogram("biases", b3)
            
            return fc


In [47]:
def main():
    # Starting a new tensorflow graph and session
    tf.reset_default_graph()
    sess = tf.Session()
    
    # Placeholder for input
    x = tf.placeholder(tf.float32, shape=[None, HEIGHT, WIDTH], name="input")
    #None means it has no specific shape, it will accept all samples given to it
    
    # Placeholder for label
    y = tf.placeholder(tf.float32, shape=[None, NUM_LABELS], name="label")
    
    # Placeholder for dropout probability, to increase accuracy during training
    dropout = tf.placeholder(tf.float32, name='dropout')
    
    # NN model
    logits = get_model(x, dropout)
    
    # Loss function
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
        tf.summary.scalar('loss', loss)
    
    #Optimizer for the loss
    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)
    
    # Accuracy Function
    with tf.name_scope("accuracy"):
        predicted = tf.argmax(logits, 1) #Returns the index with the largest value across axes of a tensor
        truth = tf.argmax(y, 1) #Returns the index with the largest value across axes of a tensor
        correct_prediction = tf.equal(predicted, truth) # tf.equal Returns the truth value of (x == y) element-wise.
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))# tf.cast 'll cast it to dtype specified, 
                                                                          #tf.rduce_mean........
        confusion_matrix = tf.confusion_matrix(truth, predicted, num_classes=NUM_LABELS)
        tf.summary.scalar("accuracy", accuracy)
        
        # Setting up Tensorboard
        summ = tf.summary.merge_all()
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer()) #tf.glb_varbls_initzer Returns an Op that initializes a list of vables.
        writer = tf.summary.FileWriter(LOGDIR) #FileWriter Create an event into the given firectory...... CHECK YOUR NOTE FOR DETAILS
        writer.add_graph(sess.graph)
        test_writer = tf.summary.FileWriter(TEST_LOGDIR)
        
        # Training model
        print('starting training\n')
        batch = get_batch(BATCH_SIZE, PATH_TRAIN)
        start_time = time.time()
        for i in range(1, ITERATIONS + 1):
            X, Y = next(batch)
            if i % EVAL_EVERY == 0:
                [train_accuracy, train_loss, s] = sess.run([accuracy, loss, summ], feed_dict={x: X, y: Y, dropout:1.0})
                acc_and_loss = [i, train_loss, train_accuracy * 100]
                print('Iteration # {}. Train Loss: {:.2f}. Train Acc: {:.0f}%'.format(*acc_and_loss))
                writer.add_summary(s,i)
                if i % (EVAL_EVERY * 20) == 0:
                    train_confusion_matrix = sess.run([confusion_matrix], feed_dict={x:X, y:Y, dropout:0.5})
                    header=LABEL_TO_INDEX_MAP.keys()
                    df=pd.DataFrame(np.reshape(train_confusion_matrix, (NUM_LABELS, NUM_LABELS)), index=header, columns=LABEL_TO_INDEX_MAP)
                    print(confusion_matrix)

                    print('\nConfusion Matrix:\n {}\n'.format(df))
                    saver.save(sess, os.path.join(LOGDIR, "model.ckpt"), i)
                                    
                sess.run(train_step, feed_dict={x:X, y:Y, dropout:0.5})
                
            #print ('\nTotal training time: {:0f} seconds\n'.format(time.time() - start_time))
            
            
            # Testing Model
            #batch = get_batch(BATCH_SIZE, PATH_TEST)
            #total_accuracy = 0
            #for i in range(ITERATIONS_TEST):
               # X, Y = next(batch, PATH_TEST)
                #test_accuracy, s = sess.run([accuracy, summ], feed_dict={x:X, y:Y, dropout:1.0})
                #print('Iteration # {}. Test Accuracy: {:.0f}%'.format(i+1, test_accuracy * 100))
                #total_accuracy +=(test_accuracy / ITERATIONS_TEST)
                #test_writer.add_summary(s, i)
                
                #print('\nFinal Test Accuracy: {:.0f}%'.format(total_accuracy * 100))

In [48]:
if __name__ == '__main__':
    init(PATH_TRAIN)
    main()

starting training

Iteration # 5. Train Loss: 2.10. Train Acc: 14%
Iteration # 10. Train Loss: 1.81. Train Acc: 20%
Iteration # 15. Train Loss: 1.75. Train Acc: 24%
Iteration # 20. Train Loss: 1.70. Train Acc: 25%
Iteration # 25. Train Loss: 1.65. Train Acc: 25%
Iteration # 30. Train Loss: 1.60. Train Acc: 36%
Iteration # 35. Train Loss: 1.67. Train Acc: 28%
Iteration # 40. Train Loss: 1.65. Train Acc: 18%
Iteration # 45. Train Loss: 1.64. Train Acc: 30%
Iteration # 50. Train Loss: 1.58. Train Acc: 40%
Iteration # 55. Train Loss: 1.56. Train Acc: 36%
Iteration # 60. Train Loss: 1.61. Train Acc: 29%
Iteration # 65. Train Loss: 1.59. Train Acc: 37%
Iteration # 70. Train Loss: 1.60. Train Acc: 43%
Iteration # 75. Train Loss: 1.61. Train Acc: 43%
Iteration # 80. Train Loss: 1.57. Train Acc: 45%
Iteration # 85. Train Loss: 1.61. Train Acc: 48%
Iteration # 90. Train Loss: 1.65. Train Acc: 38%
Iteration # 95. Train Loss: 1.59. Train Acc: 45%
Iteration # 100. Train Loss: 1.67. Train Acc: 34%
T

Iteration # 605. Train Loss: 0.65. Train Acc: 84%
Iteration # 610. Train Loss: 0.81. Train Acc: 76%
Iteration # 615. Train Loss: 0.76. Train Acc: 76%
Iteration # 620. Train Loss: 0.68. Train Acc: 83%
Iteration # 625. Train Loss: 0.65. Train Acc: 87%
Iteration # 630. Train Loss: 0.67. Train Acc: 82%
Iteration # 635. Train Loss: 0.68. Train Acc: 84%
Iteration # 640. Train Loss: 0.56. Train Acc: 89%
Iteration # 645. Train Loss: 0.74. Train Acc: 81%
Iteration # 650. Train Loss: 0.76. Train Acc: 82%
Iteration # 655. Train Loss: 0.76. Train Acc: 81%
Iteration # 660. Train Loss: 0.70. Train Acc: 78%
Iteration # 665. Train Loss: 0.82. Train Acc: 74%
Iteration # 670. Train Loss: 0.73. Train Acc: 82%
Iteration # 675. Train Loss: 0.73. Train Acc: 77%
Iteration # 680. Train Loss: 0.63. Train Acc: 87%
Iteration # 685. Train Loss: 0.59. Train Acc: 89%
Iteration # 690. Train Loss: 0.71. Train Acc: 83%
Iteration # 695. Train Loss: 0.67. Train Acc: 86%
Iteration # 700. Train Loss: 0.59. Train Acc: 91%


Iteration # 1205. Train Loss: 0.41. Train Acc: 93%
Iteration # 1210. Train Loss: 0.46. Train Acc: 87%
Iteration # 1215. Train Loss: 0.44. Train Acc: 88%
Iteration # 1220. Train Loss: 0.41. Train Acc: 92%
Iteration # 1225. Train Loss: 0.40. Train Acc: 89%
Iteration # 1230. Train Loss: 0.36. Train Acc: 94%
Iteration # 1235. Train Loss: 0.34. Train Acc: 94%
Iteration # 1240. Train Loss: 0.38. Train Acc: 91%
Iteration # 1245. Train Loss: 0.48. Train Acc: 87%
Iteration # 1250. Train Loss: 0.41. Train Acc: 91%
Iteration # 1255. Train Loss: 0.47. Train Acc: 86%
Iteration # 1260. Train Loss: 0.41. Train Acc: 92%
Iteration # 1265. Train Loss: 0.40. Train Acc: 91%
Iteration # 1270. Train Loss: 0.43. Train Acc: 89%
Iteration # 1275. Train Loss: 0.48. Train Acc: 87%
Iteration # 1280. Train Loss: 0.31. Train Acc: 97%
Iteration # 1285. Train Loss: 0.43. Train Acc: 94%
Iteration # 1290. Train Loss: 0.39. Train Acc: 88%
Iteration # 1295. Train Loss: 0.46. Train Acc: 87%
Iteration # 1300. Train Loss: 0

Iteration # 1805. Train Loss: 0.34. Train Acc: 92%
Iteration # 1810. Train Loss: 0.31. Train Acc: 93%
Iteration # 1815. Train Loss: 0.25. Train Acc: 95%
Iteration # 1820. Train Loss: 0.31. Train Acc: 95%
Iteration # 1825. Train Loss: 0.23. Train Acc: 96%
Iteration # 1830. Train Loss: 0.35. Train Acc: 92%
Iteration # 1835. Train Loss: 0.36. Train Acc: 90%
Iteration # 1840. Train Loss: 0.30. Train Acc: 93%
Iteration # 1845. Train Loss: 0.36. Train Acc: 89%
Iteration # 1850. Train Loss: 0.34. Train Acc: 92%
Iteration # 1855. Train Loss: 0.30. Train Acc: 94%
Iteration # 1860. Train Loss: 0.39. Train Acc: 88%
Iteration # 1865. Train Loss: 0.31. Train Acc: 95%
Iteration # 1870. Train Loss: 0.37. Train Acc: 90%
Iteration # 1875. Train Loss: 0.37. Train Acc: 90%
Iteration # 1880. Train Loss: 0.33. Train Acc: 94%
Iteration # 1885. Train Loss: 0.35. Train Acc: 92%
Iteration # 1890. Train Loss: 0.32. Train Acc: 92%
Iteration # 1895. Train Loss: 0.31. Train Acc: 91%
Iteration # 1900. Train Loss: 0

Iteration # 2405. Train Loss: 0.20. Train Acc: 96%
Iteration # 2410. Train Loss: 0.23. Train Acc: 97%
Iteration # 2415. Train Loss: 0.20. Train Acc: 97%
Iteration # 2420. Train Loss: 0.21. Train Acc: 96%
Iteration # 2425. Train Loss: 0.24. Train Acc: 94%
Iteration # 2430. Train Loss: 0.20. Train Acc: 100%
Iteration # 2435. Train Loss: 0.20. Train Acc: 97%
Iteration # 2440. Train Loss: 0.21. Train Acc: 96%
Iteration # 2445. Train Loss: 0.22. Train Acc: 96%
Iteration # 2450. Train Loss: 0.27. Train Acc: 95%
Iteration # 2455. Train Loss: 0.20. Train Acc: 97%
Iteration # 2460. Train Loss: 0.25. Train Acc: 95%
Iteration # 2465. Train Loss: 0.25. Train Acc: 93%
Iteration # 2470. Train Loss: 0.21. Train Acc: 95%
Iteration # 2475. Train Loss: 0.23. Train Acc: 93%
Iteration # 2480. Train Loss: 0.27. Train Acc: 95%
Iteration # 2485. Train Loss: 0.30. Train Acc: 93%
Iteration # 2490. Train Loss: 0.30. Train Acc: 89%
Iteration # 2495. Train Loss: 0.26. Train Acc: 94%
Iteration # 2500. Train Loss: 

...

Iteration # 7205. Train Loss: 0.04. Train Acc: 99%
Iteration # 7210. Train Loss: 0.02. Train Acc: 100%
Iteration # 7215. Train Loss: 0.03. Train Acc: 100%
Iteration # 7220. Train Loss: 0.05. Train Acc: 100%
Iteration # 7225. Train Loss: 0.03. Train Acc: 100%
Iteration # 7230. Train Loss: 0.04. Train Acc: 99%
Iteration # 7235. Train Loss: 0.04. Train Acc: 100%
Iteration # 7240. Train Loss: 0.04. Train Acc: 100%
Iteration # 7245. Train Loss: 0.04. Train Acc: 100%
Iteration # 7250. Train Loss: 0.06. Train Acc: 100%
Iteration # 7255. Train Loss: 0.02. Train Acc: 100%
Iteration # 7260. Train Loss: 0.03. Train Acc: 100%
Iteration # 7265. Train Loss: 0.05. Train Acc: 99%
Iteration # 7270. Train Loss: 0.05. Train Acc: 99%
Iteration # 7275. Train Loss: 0.03. Train Acc: 100%
Iteration # 7280. Train Loss: 0.03. Train Acc: 100%
Iteration # 7285. Train Loss: 0.04. Train Acc: 99%
Iteration # 7290. Train Loss: 0.05. Train Acc: 100%
Iteration # 7295. Train Loss: 0.03. Train Acc: 99%
Iteration # 7300. 

Iteration # 7805. Train Loss: 0.03. Train Acc: 100%
Iteration # 7810. Train Loss: 0.04. Train Acc: 100%
Iteration # 7815. Train Loss: 0.02. Train Acc: 100%
Iteration # 7820. Train Loss: 0.04. Train Acc: 100%
Iteration # 7825. Train Loss: 0.04. Train Acc: 100%
Iteration # 7830. Train Loss: 0.05. Train Acc: 100%
Iteration # 7835. Train Loss: 0.02. Train Acc: 100%
Iteration # 7840. Train Loss: 0.03. Train Acc: 100%
Iteration # 7845. Train Loss: 0.02. Train Acc: 100%
Iteration # 7850. Train Loss: 0.03. Train Acc: 100%
Iteration # 7855. Train Loss: 0.05. Train Acc: 99%
Iteration # 7860. Train Loss: 0.04. Train Acc: 100%
Iteration # 7865. Train Loss: 0.03. Train Acc: 100%
Iteration # 7870. Train Loss: 0.05. Train Acc: 99%
Iteration # 7875. Train Loss: 0.03. Train Acc: 100%
Iteration # 7880. Train Loss: 0.03. Train Acc: 100%
Iteration # 7885. Train Loss: 0.02. Train Acc: 100%
Iteration # 7890. Train Loss: 0.02. Train Acc: 100%
Iteration # 7895. Train Loss: 0.03. Train Acc: 100%
Iteration # 79

Iteration # 8405. Train Loss: 0.03. Train Acc: 100%
Iteration # 8410. Train Loss: 0.02. Train Acc: 100%
Iteration # 8415. Train Loss: 0.02. Train Acc: 100%
Iteration # 8420. Train Loss: 0.02. Train Acc: 100%
Iteration # 8425. Train Loss: 0.03. Train Acc: 100%
Iteration # 8430. Train Loss: 0.02. Train Acc: 100%
Iteration # 8435. Train Loss: 0.04. Train Acc: 99%
Iteration # 8440. Train Loss: 0.02. Train Acc: 100%
Iteration # 8445. Train Loss: 0.01. Train Acc: 100%
Iteration # 8450. Train Loss: 0.02. Train Acc: 100%
Iteration # 8455. Train Loss: 0.02. Train Acc: 100%
Iteration # 8460. Train Loss: 0.05. Train Acc: 99%
Iteration # 8465. Train Loss: 0.02. Train Acc: 100%
Iteration # 8470. Train Loss: 0.04. Train Acc: 100%
Iteration # 8475. Train Loss: 0.02. Train Acc: 100%
Iteration # 8480. Train Loss: 0.03. Train Acc: 100%
Iteration # 8485. Train Loss: 0.03. Train Acc: 99%
Iteration # 8490. Train Loss: 0.01. Train Acc: 100%
Iteration # 8495. Train Loss: 0.01. Train Acc: 100%
Iteration # 850

Iteration # 9005. Train Loss: 0.02. Train Acc: 100%
Iteration # 9010. Train Loss: 0.02. Train Acc: 100%
Iteration # 9015. Train Loss: 0.02. Train Acc: 100%
Iteration # 9020. Train Loss: 0.02. Train Acc: 100%
Iteration # 9025. Train Loss: 0.02. Train Acc: 100%
Iteration # 9030. Train Loss: 0.01. Train Acc: 100%
Iteration # 9035. Train Loss: 0.02. Train Acc: 100%
Iteration # 9040. Train Loss: 0.03. Train Acc: 100%
Iteration # 9045. Train Loss: 0.02. Train Acc: 100%
Iteration # 9050. Train Loss: 0.02. Train Acc: 100%
Iteration # 9055. Train Loss: 0.01. Train Acc: 100%
Iteration # 9060. Train Loss: 0.01. Train Acc: 100%
Iteration # 9065. Train Loss: 0.03. Train Acc: 99%
Iteration # 9070. Train Loss: 0.01. Train Acc: 100%
Iteration # 9075. Train Loss: 0.03. Train Acc: 100%
Iteration # 9080. Train Loss: 0.02. Train Acc: 100%
Iteration # 9085. Train Loss: 0.02. Train Acc: 100%
Iteration # 9090. Train Loss: 0.03. Train Acc: 99%
Iteration # 9095. Train Loss: 0.03. Train Acc: 100%
Iteration # 91

Iteration # 9605. Train Loss: 0.01. Train Acc: 100%
Iteration # 9610. Train Loss: 0.02. Train Acc: 100%
Iteration # 9615. Train Loss: 0.02. Train Acc: 100%
Iteration # 9620. Train Loss: 0.02. Train Acc: 100%
Iteration # 9625. Train Loss: 0.02. Train Acc: 100%
Iteration # 9630. Train Loss: 0.01. Train Acc: 100%
Iteration # 9635. Train Loss: 0.02. Train Acc: 100%
Iteration # 9640. Train Loss: 0.01. Train Acc: 100%
Iteration # 9645. Train Loss: 0.02. Train Acc: 100%
Iteration # 9650. Train Loss: 0.04. Train Acc: 99%
Iteration # 9655. Train Loss: 0.02. Train Acc: 100%
Iteration # 9660. Train Loss: 0.02. Train Acc: 100%
Iteration # 9665. Train Loss: 0.01. Train Acc: 100%
Iteration # 9670. Train Loss: 0.02. Train Acc: 100%
Iteration # 9675. Train Loss: 0.02. Train Acc: 100%
Iteration # 9680. Train Loss: 0.02. Train Acc: 100%
Iteration # 9685. Train Loss: 0.02. Train Acc: 100%
Iteration # 9690. Train Loss: 0.02. Train Acc: 100%
Iteration # 9695. Train Loss: 0.03. Train Acc: 99%
Iteration # 97